In [8]:
# utils_display_records.py

from dotenv import load_dotenv
import os
from sqlalchemy import create_engine, MetaData

load_dotenv()

db_user = os.getenv("DB_USER")
db_pass = os.getenv("DB_PASSWORD")
db_host = os.getenv("DB_HOST", "localhost")
db_port = os.getenv("DB_PORT", "5432")
db_name = os.getenv("DB_NAME", "dag_review_db")

db_url = f"postgresql+psycopg2://{db_user}:{db_pass}@{db_host}:{db_port}/{db_name}"
engine = create_engine(db_url)
metadata = MetaData()
metadata.reflect(bind=engine)

def display_all_records():
    with engine.connect() as conn:
        print("\n--- Studies ---")
        studies = conn.execute(metadata.tables['studies'].select()).fetchall()
        for row in studies:
            print(dict(row._mapping))

        print("\n--- Objectives ---")
        objectives = conn.execute(metadata.tables['objectives'].select()).fetchall()
        for row in objectives:
            print(dict(row._mapping))

        print("\n--- Eligibility Criteria ---")
        eligibility = conn.execute(metadata.tables['eligibility_criteria'].select()).fetchall()
        for row in eligibility:
            print(dict(row._mapping))

        print("\n--- Outcomes ---")
        outcomes = conn.execute(metadata.tables['outcomes'].select()).fetchall()
        for row in outcomes:
            print(dict(row._mapping))

        print("\n--- Exposures ---")
        exposures = conn.execute(metadata.tables['exposures'].select()).fetchall()
        for row in exposures:
            print(dict(row._mapping))

        print("\n--- Extraction Reviews ---")
        reviews = conn.execute(metadata.tables['extraction_reviews'].select()).fetchall()
        for row in reviews:
            print(dict(row._mapping))

In [9]:
display_all_records()


--- Studies ---
{'id': 2, 'title': 'Physical Activity and Cognitive Decline', 'authors': 'Smith et al.', 'doi': '10.1234/example.doi', 'source': 'Mock Text', 'year': 2024, 'llm_model_used': 'gpt-4', 'date_processed': datetime.datetime(2025, 5, 20, 23, 5, 51, 231307)}

--- Objectives ---
{'id': 2, 'study_id': 2, 'content': 'To identify the prevalence of stage B heart failure (SBHF) in patients with type 2 diabetes mellitus (T2DM) with no history of cardiovascular disease (CVD).', 'type': 'primary', 'confidence_score': 1.0, 'validated': False}

--- Eligibility Criteria ---
{'id': 3, 'study_id': 2, 'criteria': 'Patients with type 2 diabetes mellitus (T2DM) with no history of cardiovascular disease (CVD)', 'inclusion': True, 'validated': False}

--- Outcomes ---
{'id': 2, 'study_id': 2, 'data': [{'type': 'primary', 'value': 'Echocardiographic parameters such as left ventricular ejection fraction, left ventricular mass index (LVMI), left ventricular hypertrophy, left atrial enlargement and

In [19]:
import difflib
import json

def summarize_study_reviews():
    with engine.connect() as conn:
        studies = conn.execute(metadata.tables['studies'].select()).fetchall()
        for study in studies:
            study_id = study._mapping['id']
            title = study._mapping['title']
            print(f"\n=== Study: {title} (ID: {study_id}) ===")

            # Objectives
            objectives = conn.execute(metadata.tables['objectives'].select().where(
                metadata.tables['objectives'].c.study_id == study_id)).fetchall()
            print("- Objectives:")
            for obj in objectives:
                print(f"  • {obj._mapping['content']}")

            # Outcomes
            outcomes = conn.execute(metadata.tables['outcomes'].select().where(
                metadata.tables['outcomes'].c.study_id == study_id)).fetchall()
            print("- Outcomes:")
            for out in outcomes:
                print(f"  • {out._mapping['data']}")

            # Eligibility
            eligibility = conn.execute(metadata.tables['eligibility_criteria'].select().where(
                metadata.tables['eligibility_criteria'].c.study_id == study_id)).fetchall()
            print("- Eligibility:")
            for crit in eligibility:
                flag = "Include" if crit._mapping['inclusion'] else "Exclude"
                print(f"  • {flag}: {crit._mapping['criteria']}")

            # Diff
            review = conn.execute(metadata.tables['extraction_reviews'].select().where(
                metadata.tables['extraction_reviews'].c.study_id == study_id)).fetchone()
            if review:
                print("- Diff (LLM vs User JSON):")
                raw = review._mapping['raw_llm_json']
                user = review._mapping['user_json']
                if isinstance(raw, str):
                    raw_json = raw
                else:
                    raw_json = json.dumps(raw, indent=2)
                if isinstance(user, str):
                    user_json = user
                else:
                    user_json = json.dumps(user, indent=2)
                # Enhanced diff output
                added = []
                removed = []
                updated = []

                raw_dict = json.loads(raw_json) if isinstance(raw_json, str) else raw_json
                user_dict = json.loads(user_json) if isinstance(user_json, str) else user_json

                all_keys = set(raw_dict.keys()).union(user_dict.keys())

                for key in all_keys:
                    raw_val = raw_dict.get(key)
                    user_val = user_dict.get(key)
                    if raw_val == user_val:
                        continue
                    elif raw_val is None:
                        added.append((key, user_val))
                    elif user_val is None:
                        removed.append((key, raw_val))
                    else:
                        updated.append((key, raw_val, user_val))

                if added:
                    print("  == Added ==")
                    for k, v in added:
                        print(f"    + {k}: {json.dumps(v, indent=2)}")
                if removed:
                    print("  == Removed ==")
                    for k, v in removed:
                        print(f"    - {k}: {json.dumps(v, indent=2)}")
                if updated:
                    print("  == Updated (original shown) ==")
                    for k, old, new in updated:
                        print(f"    ~ {k}:\nFrom: {json.dumps(old, indent=2)}\nTo:   {json.dumps(new, indent=2)}")



In [20]:
summarize_study_reviews()


=== Study: Physical Activity and Cognitive Decline (ID: 2) ===
- Objectives:
  • To identify the prevalence of stage B heart failure (SBHF) in patients with type 2 diabetes mellitus (T2DM) with no history of cardiovascular disease (CVD).
- Outcomes:
  • [{'type': 'primary', 'value': 'Echocardiographic parameters such as left ventricular ejection fraction, left ventricular mass index (LVMI), left ventricular hypertrophy, left atrial enlargement and diastolic function'}]
- Eligibility:
  • Include: Patients with type 2 diabetes mellitus (T2DM) with no history of cardiovascular disease (CVD)
- Diff (LLM vs User JSON):
  == Updated (original shown) ==
    ~ eligibility:
From: [
  {
    "criteria": "Patients with type 2 diabetes mellitus (T2DM) with no history of cardiovascular disease (CVD)",
    "inclusion": true
  },
  {
    "criteria": "Patients with deranged liver function tests and end stage renal failure",
    "inclusion": false
  }
]
To:   [
  {
    "criteria": "Patients with type 